Credit goes to https://colab.research.google.com/github/count0/colab-gt/blob/master/colab-gt.ipynb for the steps to use graph_tool in Colab.

Install graph_tool

In [1]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

Executing: /tmp/apt-key-gpghome.wIhlYqheWA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://downloads.skewed.de/apt jammy InRelease [7,535 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,463 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,796 kB]
Hit:10 https://ppa.launchpadconte

Perform a one time clean-up on the network.

In [2]:
import networkx as nx

try:
    open("clean-HepPh.graphml", "r").close()
except:
    # Load edges
    G = nx.read_edgelist("cit-HepPh.txt", nodetype=int, create_using=nx.DiGraph)

    # Verify dataset is correct
    print("Verifiying dataset correctness")
    print("Number of nodes:", G.order())
    print("Number of edges:", G.size())
    print()

    # Remove self loops
    G.remove_edges_from(nx.selfloop_edges(G))
    print("After removing self-loops")
    print("Number of nodes:", G.order())
    print("Number of edges:", G.size())
    print()

    # Load dates
    dates = {}
    with open("cit-HepPh-dates.txt") as f:
        for line in f:
            if line[0] == "#":
                continue
            nodeID = line.split()[0]
            if nodeID[:2]=="11":
                nodeID = int(nodeID[2:])
            dates[int(nodeID)] = line.split()[1]

    nodes_to_delete = []
    for nodeID in G.nodes:
        try:
            G.nodes[nodeID]["date"] = dates[nodeID]
        except KeyError:
            nodes_to_delete.append(nodeID)
    G.remove_nodes_from(nodes_to_delete)
    print("After removing nodes without dates")
    print("Number of nodes:", G.order())
    print("Number of edges:", G.size())
    print()

    edges_to_delete = []
    for u,v in G.edges():
        if G.nodes[u]["date"] < G.nodes[v]["date"]:
            edges_to_delete.append((u,v))
    G.remove_edges_from(edges_to_delete)
    print("After removing edges from past to future")
    print("Number of nodes:", G.order())
    print("Number of edges:", G.size())
    print()

    print("Finally")
    print("Number of papers:", G.order())
    print("Number of edges:", G.size())
    print("Number of phantom papers (now removed):", len(nodes_to_delete))

    nx.write_graphml_lxml(nx.convert_node_labels_to_integers(G, label_attribute="arxiv_id"), "temp_HepPh.graphml")
    G.clear()

Verifiying dataset correctness
Number of nodes: 34546
Number of edges: 421578

After removing self-loops
Number of nodes: 34546
Number of edges: 421534

After removing nodes without dates
Number of nodes: 30561
Number of edges: 347288

After removing edges from past to future
Number of nodes: 30561
Number of edges: 345564

Finally
Number of papers: 30561
Number of edges: 345564
Number of phantom papers (now removed): 3985


In [3]:
from graph_tool.all import *

G = load_graph("temp_HepPh.graphml")

# Clean up and remove cyclic dependencies by using the preprint transformation
# https://arxiv.org/pdf/cs/0309023.pdf
comp, hist = label_components(G)
for sccIdx, sccNumNodes in enumerate(hist):
    if sccNumNodes == 2:
        v1preprint = G.add_vertex()
        v2preprint = G.add_vertex()
        count = 0
        node1 = 0
        node2 = 0
        for nodeIdx, sccNum in enumerate(comp.a):
            if sccNum == sccIdx:
                G.add_edge(G.vertex(nodeIdx), v1preprint)
                G.add_edge(G.vertex(nodeIdx), v2preprint)
                if count == 0:
                    G.vertex_properties.date[v1preprint] = G.vertex_properties.date[G.vertex(nodeIdx)]
                    node1 = nodeIdx
                    count += 1
                else:
                    G.vertex_properties.date[v2preprint] = G.vertex_properties.date[G.vertex(nodeIdx)]
                    node2 = nodeIdx
                    break
        G.remove_edge(G.edge(node1, node2))
        G.remove_edge(G.edge(node2, node1))
    elif sccNumNodes == 3:
        v1preprint = G.add_vertex()
        v2preprint = G.add_vertex()
        v3preprint = G.add_vertex()
        count = 0
        node1 = 0
        node2 = 0
        node3 = 0
        for nodeIdx, sccNum in enumerate(comp.a):
            if sccNum == sccIdx:
                G.add_edge(G.vertex(nodeIdx), v1preprint)
                G.add_edge(G.vertex(nodeIdx), v2preprint)
                G.add_edge(G.vertex(nodeIdx), v3preprint)
                if count == 0:
                    G.vertex_properties.date[v1preprint] = G.vertex_properties.date[G.vertex(nodeIdx)]
                    node1 = nodeIdx
                    count += 1
                elif count == 1:
                    G.vertex_properties.date[v2preprint] = G.vertex_properties.date[G.vertex(nodeIdx)]
                    node2 = nodeIdx
                    count += 1
                else:
                    G.vertex_properties.date[v3preprint] = G.vertex_properties.date[G.vertex(nodeIdx)]
                    node3 = nodeIdx
                    break
        if G.edge(node1, node2) != None:
            G.remove_edge(G.edge(node1, node2))
        if G.edge(node2, node1) != None:
            G.remove_edge(G.edge(node2, node1))
        if G.edge(node1, node3) != None:
            G.remove_edge(G.edge(node1, node3))
        if G.edge(node3, node1) != None:
            G.remove_edge(G.edge(node3, node1))
        if G.edge(node2, node3) != None:
            G.remove_edge(G.edge(node2, node3))
        if G.edge(node3, node2) != None:
            G.remove_edge(G.edge(node3, node2))

# Verify transformation is valid
assert(is_DAG(G))

print("After preprint transform")
print("Number of nodes:", G.num_vertices())
print("Number of edges:", G.num_edges())

# Extract the largest component
G_largest = extract_largest_component(G, directed=False, prune=True)

# Save this new version of graph, we are finally done with preprocessing.
G_largest.save("cleaned_HepPh.graphml")
print("After extracting largest component")
print("Number of nodes:", G_largest.num_vertices())
print("Number of edges:", G_largest.num_edges())

After preprint transform
Number of nodes: 30659
Number of edges: 345664
After extracting largest component
Number of nodes: 30438
Number of edges: 345573
